In [1]:
import bdkpython as bdk
# network
net = bdk.Network.REGTEST

In [2]:
# generate mnemonic from entropy
entropy = b"q"*16
mnemonic = bdk.Mnemonic.from_entropy(entropy)
mnemonic.as_string()

'imitate merit fox title blade reward shift comfort imitate merit fox tip'

In [3]:
# calculate root private key
root = bdk.DescriptorSecretKey(net, mnemonic, None)
root.as_string()

'tprv8ZgxMBicQKsPf1D2Rt3JBKp8KEpMiKTDxaRiBHJZ4YtTxhtgj3XCEifvBewPeN5GXTtZS7S4NFmFSWhKg2g1EhsHeXgYPdmyeYMLWW1ywH4/*'

In [4]:
# wallet name
wname = "test"

# database config. available: MEMORY, SQLITE(path), SLED

# In-memory database:
db_config = bdk.DatabaseConfig.MEMORY()

# SQLite database:
# db_config = bdk.DatabaseConfig.SQLITE(
#     bdk.SqliteDbConfiguration(
#         f"{wname}.sqlite" # path to sqlite file
#     )
# )

# SLED database:
# db_config = bdk.DatabaseConfig.SLED(
#     bdk.SledDbConfiguration(
#         f"wallets", # path to database
#         wname # tree name (wallet name for example)
#     )
# )

# Electrum backend
# blockchain_config = bdk.BlockchainConfig.ELECTRUM(
#     bdk.ElectrumConfig(
#         "ssl://electrum.blockstream.info:60002", # url
#         None, # socks5
#         5, # retry
#         None, # timeout
#         100, # stop gap
#         True, # validate domain
#     )
# )

# Bitcoin Core backend
blockchain_config = bdk.BlockchainConfig.RPC(
    bdk.RpcConfig(
        "127.0.0.1:18443", # url
        bdk.Auth.USER_PASS("bitcoin","secret"), #auth
        net, # network
        wname, # Bitcoin Core wallet
        None, # sync params (https://docs.rs/bdk/latest/bdk/blockchain/rpc/struct.RpcSyncParams.html)
    )
)

blockchain = bdk.Blockchain(blockchain_config)


bdkpython.bdk.BlockchainConfig.ESPLORA

In [5]:
# create signer
k1 = root.derive(bdk.DerivationPath("m/84h/1h/0h/0"))
k2 = root.derive(bdk.DerivationPath("m/84h/1h/0h/1"))
pdesc1 = bdk.Descriptor(f"wpkh({k1.as_string()})", net)
pdesc2 = bdk.Descriptor(f"wpkh({k2.as_string()})", net)
signer = bdk.Wallet(
    descriptor=pdesc1,
    change_descriptor=pdesc2,
    network=net,
    database_config=db_config
)

In [6]:
# create receive and change descriptors:
acc = root.derive(bdk.DerivationPath("m/84h/1h/0h"))
xpub = acc.as_public().as_string().replace("/*","")
recv_descriptor = bdk.Descriptor(f"wpkh({xpub}/0/*)", net)
change_descriptor = bdk.Descriptor(f"wpkh({xpub}/1/*)", net)

recv_descriptor.as_string(), change_descriptor.as_string()

("wpkh([6d771ec1/84'/1'/0']tpubDCJbNvJTpXKSuShm4Gid6tSSNYg5mY8spTxAGBQi4CfoDVc8oJFZgPMaC4RyiLeVP8U5tzoYYWGtaBfMTiLyRSzT1tcxSq1iMYCPt13aNie/0/*)#43rxnv36",
 "wpkh([6d771ec1/84'/1'/0']tpubDCJbNvJTpXKSuShm4Gid6tSSNYg5mY8spTxAGBQi4CfoDVc8oJFZgPMaC4RyiLeVP8U5tzoYYWGtaBfMTiLyRSzT1tcxSq1iMYCPt13aNie/1/*)#y9x8wepz")

In [7]:
wallet = bdk.Wallet(
    descriptor=recv_descriptor,
    change_descriptor=change_descriptor,
    network=net,
    database_config=db_config
)
address_info = wallet.get_address(bdk.AddressIndex.LAST_UNUSED)
address = address_info.address
print(address_info)

AddressInfo(index=0, address=bcrt1qt662q8wcu75fkna2ftyktkyn9wynp736aktkkr)


In [8]:
wallet.sync(blockchain, None)
balance = wallet.get_balance()

In [9]:
balance.confirmed

98999828

In [10]:
wallet.get_address(bdk.AddressIndex(2)).address

'bcrt1q7w94rp03v0fyx9f5cdyd4wne8zze0n08ztzhsz'

In [ ]:
# send 1M sat to bcrt1q6yyrmlxllr3msxwc562s6hwa9ah2p00spmveat
addr = bdk.Address("bcrt1q6yyrmlxllr3msxwc562s6hwa9ah2p00spmveat")
# tx builder requires chaining or re-assigning
tx_builder = bdk.TxBuilder()
tx_builder = tx_builder.add_recipient(addr.script_pubkey(), 1e6)
# set fee rate in sat_per_vbyte
tx_builder = tx_builder.fee_rate(1.2)
# enable replace by fee
tx_builder = tx_builder.enable_rbf()
# specify utxos if you want
tx_builder.add_utxo(wallet.list_unspent()[0].outpoint)
# create transaction using wallet
res = tx_builder.finish(wallet)

In [ ]:
psbt1 = res.psbt

In [ ]:
# get spent amount:
res.transaction_details.sent - res.transaction_details.received

In [ ]:
res.transaction_details.fee

In [ ]:
# sign transaction - this method mutates transaction, so we copy it first
psbt2 = bdk.PartiallySignedTransaction(psbt1.serialize())
signer.sign(psbt2)

In [ ]:
combined = psbt1.combine(psbt2)
combined.serialize()

In [ ]:
# get raw transaction for broadcasting
tx = combined.extract_tx()
bytes(tx).hex()

In [ ]:
blockchain.broadcast(combined)

In [ ]:
combined.txid()

In [ ]:
txs = wallet.list_transactions()
txs

In [ ]:
tx = txs[0]

In [ ]:
tx.received, tx.sent, tx.fee, tx.txid